In [10]:
import pyaudio
import audioop
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from sklearn.neighbors import KNeighborsClassifier

In [11]:
class TapDetector():
    def __init__(this,channels,rate,index,frames,frames2,chunk,tapSize):
        this.audio = pyaudio.PyAudio()
        this.audio2 = pyaudio.PyAudio()
        this.fullFrames = np.full((tapSize,chunk),None,dtype='float64')
        this.fullFrames2 = np.full((tapSize,chunk),None,dtype='float64')
        this.frames = np.zeros(tapSize)
        this.frames2 = np.zeros(tapSize) # <--- 2
        this.all = []
        this.tapSize = tapSize
        this.currentFrame = 0
        this.chunk = chunk
        this.rate = rate
        this.cooldown = 0
        this.trainingSets = []
        this.negativeTrainingSet = []
        this.clf = None
        this.classifier = KNeighborsClassifier(3)
        this.stream = this.audio.open(format = pyaudio.paInt16,
                                      channels = channels,
                                      rate = rate,
                                      input = True,
                                      input_device_index = index,
                                      frames_per_buffer = frames)
        this.stream2 = this.audio.open(format = pyaudio.paInt16, # <--- 2
                                      channels = channels,
                                      rate = rate,
                                      input = True,
                                      input_device_index = index2,
                                      frames_per_buffer = frames2)
    
    def start(this,seconds):
        print("Recording Started")
        print(not this.clf == None)
        for i in range(0,int(this.rate / this.chunk * seconds)):
            if this.record():
                print("clf={}".format(this.clf))
                if not this.clf == None:
                    flat = np.array([np.array(x).flatten() for x in this.fullFrames]).flatten()
                    print(np.array(flat).shape)
                    prediction = this.clf.predict([flat])
                    print(prediction)
        print("Recording Finnished")
        plt.plot(this.all)
        plt.show()
    
    def record(this):
        tapFound = False
        data = this.stream.read(this.chunk, exception_on_overflow=False)
        data2 = this.stream2.read(this.chunk, exception_on_overflow=False) # <--- 2
        decoded = np.fromstring(data.hex(), 'Float32').tolist()
        decoded2 = np.fromstring(data2.hex(), 'Float32').tolist() # <--- 2
        this.all.append(audioop.rms(data, 2))
        this.all.append(audioop.rms(data2, 2)) # <--- 2
        this.frames[this.currentFrame] = audioop.rms(data, 2)
        this.fullFrames[this.currentFrame] = decoded
        this.fullFrames2[this.currentFrame] = decoded2
        this.currentFrame = (this.currentFrame+1)%this.tapSize
        if(this.cooldown <= 0):
            tapFound = this.isTap(9000)
            return tapFound
        else:
            this.cooldown -= 1
        return tapFound
    
    def callibrate(this,buttons,tapsNeeded):
        for button in range(0,buttons):
            this.trainingSets.append([])
            print("Callibrating button position {}".format(button))
            for tap in range(1,tapsNeeded+1):
                while(not this.record()):
                    d = 1
                this.trainingSets[button].append(this.fullFrames)
                print("{}/{} taps registered for button {}".format(tap,tapsNeeded,button))
        for tap in range(1,tapsNeeded+1):
            print("Tap at a place with no button")
            while(not this.record()):
                d = 1
            this.negativeTrainingSet.append(this.fullFrames)
            print("{}/{} false taps registered so far".format(tap,tapsNeeded))
        this.buttonTraining()

    def buttonTraining(this):
        positiveTrainX = [np.array(x).flatten() for x in this.trainingSets[0]]
        positiveTrainY = np.full( len(positiveTrainX), 1)
        negativeTrainX = [np.array(x).flatten() for x in this.negativeTrainingSet]
        negativeTrainY = np.full( len(negativeTrainX), 0)
        trainX = positiveTrainX
        trainX.extend(negativeTrainX)
        trainY = np.hstack([positiveTrainY,negativeTrainY])
        #print(positiveTrainX[2],negativeTrainX[0])
        print(np.array(positiveTrainX).shape)
        print(np.array(trainX[0]).shape)
        print(np.array(trainY).shape)
        #print(trainX.shape,trainY.shape)
        this.clf = this.classifier.fit(trainX,trainY)
        
    def stop(this):
        this.stream.stop_stream()
        this.stream.close()
        this.stream2.stop_stream()
        this.stream2.close()
    
    def plot(this):
        plt.plot(np.hstack([this.frames[this.currentFrame:],this.frames[:this.currentFrame]]))
        plt.plot(np.hstack([this.frames2[this.currentFrame:],this.frames2[:this.currentFrame]]))
        #plt.show()
    
    def isTap(this,threshold):
        soundSlice = np.hstack([this.frames[this.currentFrame:],this.frames[:this.currentFrame]])
        soundSlice2 = np.hstack([this.frames2[this.currentFrame:],this.frames2[:this.currentFrame]])
        diff = np.diff(soundSlice)
        diff2 = np.diff(soundSlice2)
        maxPeaks = argrelextrema(soundSlice, np.greater)
        maxPeaks2 = argrelextrema(soundSlice2, np.greater)
        if len(maxPeaks) > 0 or len(maxPeaks2) > 0: # <--- 2, maybe needs more changes?..
            maxPeaks = maxPeaks[0].tolist()
            maxPeaks2 = maxPeaks2[0].tolist()
        if len(maxPeaks) == 0 and len(maxPeaks2) == 0 :
            return False
        if (soundSlice[maxPeaks[0]] > threshold and diff[maxPeaks[0]]**2 > 5000 and maxPeaks[0] > 1) or (
            soundSlice2[maxPeaks2[0]] > threshold and diff2[maxPeaks[0]]**2 > 5000 and maxPeaks2[0] > 1):
            for i in range(1,len(maxPeaks)):
                if (soundSlice[maxPeaks[i]] > (soundSlice[maxPeaks[0]] * 0.5)) or (
                   soundSlice2[maxPeaks2[i]] > (soundSlice2[maxPeaks2[0]] * 0.5)):
                    return False
            this.cooldown = 50
            print("found a tap")
            this.plot()
            return True
        return False
                   
    

In [12]:
channels = 1
rate = 100000 #44100
index = 2
index2 = 6
frames = 5
frames2 = 5
chunk = 1024
tapSize = 25
detector = TapDetector(channels,rate,index,frames,frames2,chunk,tapSize)
detector.callibrate(1,20)
detector.start(10)
detector.plot()
detector.stop()

Callibrating button position 0


c:\python37\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
c:\python37\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
c:\python37\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
c:\python37\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


IndexError: list index out of range

In [ ]:
        
    def findPeaks(this,soundSlice,threshold):
        diff = np.diff(soundSlice)
        startPeak,startId = 0,0
        lowPeak,lowId = 0,0
        afterPeak, afterId = 0,0
        index = 0
        for db in soundSLice:
            index += 1
            if(lowId = 0 and db > 0 and db > startPeak and db >= threshold):
                startPeak = db
                startId = index
            elif(afterId = 0 and db < 0 and db < lowPeak):
                lowPeak = db,
                lowId = index
            elif(index > lowId and db > 0 and db > afterPeak):
                afterPeak = db
                afterId = index
            elif(after)